# Local Emulators
- Emulators can have different levels of generality
- Some emulators are tailored to a specific scenario such as a specific glacier
- Here we'll look at emulators that are aimed at being fairly general stand ins for expensive models
- These emulators take advantage of the local spatial relationships between variables of interest


# IGM
- The Instructed Glacier Model ([IGM](https://www.cambridge.org/core/journals/journal-of-glaciology/article/deep-learning-speeds-up-ice-flow-modelling-by-several-orders-of-magnitude/748E962A103D2AF45F4CA8823C88C0C0)) drastically speeds up simulations of mountain glaciers 
- Estimates ice flow velocity from geometry, basal traction coefficient, and rate factor
- Approximates ice flow velocity using a CNN
- All inputs / outputs defined on grid
- 10 vertical spatial layers

<div>
<img src="images/local_emulators/igm1.png" width="800"/>
</div>


| Variable              | Symbol | 
| :---------------- | :------: | 
| Ice Thickness       |   $h$   |
| Surface Elevation         |   $s$   | 
| Rate Factor   |  $A$   |
| Basal Traction Coefficient |  $ c$  |
| Grid Resolution |  $H$  |
| Velocity X-component | $u$ |
| Velocity Y-component | $v$ |

<div>
<img src="images/local_emulators/igm2.png" width="800"/>
</div>


## PINN Loss function
- Solving for ice velocity can be posed as minimization problem
- The varitional principle for momentum balance is given by
  $$ J(\pmb{u}) = \int_{\Omega} \left [ \frac{2n}{n+1} \eta (\dot{\epsilon}^2) \dot{\epsilon}^2 + \rho g \pmb{u} \cdot \nabla{s}   \right ] \; d \Omega + \int_{\Gamma_b} \frac{c}{m+1} (\pmb{u} \cdot \pmb{u})^{\frac{m+1}{2}} \; d \Gamma $$
- This serves as a natural way of defining a loss function for training the CNN
- In IGM tensor $\dot{\epsilon}$ is simplified using the first order approxmiation

## Training

- IGM has been geared toward simulating mountain glaciers and ice fields
- Hence training data involves simulations on mountainous terrain in the European Alps generated by CsFlow or PISM
- IGM does not need training data per se, but it does need a wide variety of input scenarios to train a generallly applicable emulator

<div>
<img src="images/local_emulators/igm_training.png" width="900"/>
</div>


# Graph Neural Network Emulator

- IGM is designed for an ice flow model designed on a uniform grid
- Many models, however, are defined on an unstructured mesh
- This motivates using an alternative approach that can handle models defined on non-uniform meshes

## Graphs
- A graph $G=(V,E)$ is a tuple consisting of a set $V$ of vertices / nodes and a set $E$ of edges

<div>
<img src="images/local_emulators/graph.png" width="700"/>
</div>

- A fine element mesh has a clear graph structure
- Mesh vertices $\to$ graph vertices
- Mesh edges $\to$ graph edges

# Graph Neural Network
- Graph neural networks (GNN's) operate on graphs
- Inputs / outpus of a GNN are defined on graphs 
- A GNN takes in a graph structure $G = (V, E)$ as well as node and edge features 
- Notation $\pmb{v_i}$ refers to a vector feature defined on node $i$
- Notation $\pmb{e_{ij}}$ refers to a vector features defined on edge connecting nodes $i$ and $j$
- Currently there are many GNN architectures for different applications
- A popular Python package for GNN's is [Pytorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) which has implementations of many different architectures

## GNN Architecture
- For physics application GNN's using graph convolutions have proved useful
- These architectures use multiple graph convolution layers analagous to CNN's
- Very informative breakdown of graph convolutions [here](https://distill.pub/2021/understanding-gnns/)
- We use a GNN architecture based on [Pfaff 2021](https://arxiv.org/pdf/2010.03409) which implements very general graph convolutions using message passing

<div>
<img src="images/local_emulators/algorithm.png" width="700"/>
</div>

# Emulating SpecEIS
- Here, we apply a GNN to emulate the SpecEIS model using a compatible finite elements [formulation](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=10221861) 
- Note a few notational difference from IGM
- SpecEIS is a first order model that solves for the depth averaged velocity $u_{bar}$ and deformational velocity $u_{def}$ 
- SpecEIS uses a combination of finite elements including:
- MTW elements for velocity, bed, and surface gradients
- DG0 elements for bed and thickness
- CG1 elements for basal traction coefficient
- Implemented in Firedrake


| Variable              | Symbol | Degree of Freedom | FEM Space |
| :---------------- | :------: | ----: | ---- |
| Bed       |   $B$   | 1 per cell | DG0 |
| Surface          |   $S$   | 1 per cell | DG0 |
| Surface gradient   |  $\nabla S$   | 3 per edge | MTW |
| Bed gradient|  $ \nabla B$   | 3 per edge | MTW |
| Basal Traction Coefficient |  $ \beta^2$   | 1 per vertex | CG1 |
| Depth Averaged Velocity       |   $u_{bar}$   | 3 per edge | MTW |
| Deformational Velocity       |   $u_{def}$   | 3 per edge | MTW |

## Graph Representation

- All of the necessary input / output variables including necessary mesh geometric variables can be represented using edge features in  a GNN!
- Feature vector for the edge connecting nodes $i$ and $j$:
$$ \pmb{e_{ij}} = \begin{bmatrix}
    H_{ij} \\
    \beta^2_{ij}\\
    \pmb{\nabla S_{ij}} \\
    \pmb{\nabla B_{ij}} \\
    \ell_{ij} \\
    \pmb{d_{ij}}^+ \\
    \pmb{d_{ij}}^- \\
\end{bmatrix}
$$

| Feature      | Description | Size |
| :----------- | :------: | ----: |
| $H_{ij}$       |  Average of $H$ over edge $i,j$ | 1 |
| $\beta^2_{ij}$       |  Average of $\beta^2$ over edge $i,j$ | 1 |
| $\nabla S_{ij}$       |  Gradient of $S$ in MTW space | 3 |
| $\nabla B_{ij}$       |  Gradient of $B$ in MTW space | 3 |
|  $\ell_{ij}$       |  Length of edge | 1 |
|   $\pmb{d_{ij}}^+$      |  Vector from edge to cell midpoint | 2 |
|   $\pmb{d_{ij}}^-$      |  Vector from edge to cell midpoint | 2 |

- Outputs of the emulator are degrees of freedom for $u_{bar}$ and $u_{def}$ in MTW space (3 per edge)
- Hence, we get a direct finite element representation of the solution that can be used in Firedrake

## Training

- GNN emulator is trained on mountainous terrain in Montana
- We run 400 year simulations on a number of mountainous regions
- Loss function integrates difference between solved and emulated glacier velocity
- Solved velcty is given by finite element solution
$$ 
\int_{\Omega} \pmb{r} \cdot \pmb{r} d \Omega = \int (\pmb{u}_{solved} - \pmb{u}_{emulated}) \cdot (\pmb{u}_{solved} - \pmb{u}_{emulated}) d \Omega
$$
